In [66]:
mylist = []

for chunk in  pd.read_csv('C:/Analytics/data/donations.csv', chunksize=20000):
    # Delete columns that are irrelevant
    # Unnamed: 0                This column is just an index
    # collected_date            Irrelevant. This is just when it was collected
    # is_offline                We don't understand what this means. Out of scope
    # profile_url               This is similar to the name. It is just linked to the name of a user
    # verified                  We aren't sure what this means. Out of scope
    # name                      Not relevant to our statistics. Just an identifier
    dropcols = ['Unnamed: 0', 'collected_date', 'is_offline', 'profile_url', 'verified', 'name']
    
    clean_data(chunk, dropcols)
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)

In [14]:
def clean_data(df, drop):
    
    # Remove rows that don't involve 2020
    year = df.created_at.str[:4]
    df['ld_year'] = year 
    df.drop(df[df['ld_year'] != '2020'].index, inplace=True)
    df.drop(columns=['ld_year'], inplace=True)
    
    for i in drop:
        df.drop(columns=[i], inplace=True)
    
    return df

In [193]:
# I'm going to make a copy of this data to play with so that if I mess it up, I don't have to re-run the CSV
play_data = big_data.copy(deep = True)

In [194]:
# Let's change our data types
play_data['created_at'] = play_data['created_at'].astype(str)
play_data['campaign_id'] = play_data['campaign_id'].astype(int)
play_data['is_anonymous'] = play_data['is_anonymous'].astype(bool)
play_data['amount'] = pd.to_numeric(play_data['amount'])
play_data['donation_id'] = play_data['donation_id'].astype(int)

In [188]:
# Let's get a summary of all our data
uniStats(play_data)

,DataType,Count,Null Count,Unique Values,Mode,Mean,STD,Min,25%,Median,75%,Max,Skew,Kurt
created_at,object,35001,0,33738,2020-03-12T17:21:45-05:00,-,-,-,-,-,-,-,-,-
amount,int64,35001,0,277,50,72.648,1130.36,5,15,25,50,198000,156.498,26977.7
donation_id,int32,35001,0,35001,31916,4.55054e+08,1.22785e+08,31916,4.87295e+08,4.8892e+08,4.89799e+08,496171348,-3.43001,9.78924
campaign_id,int32,35001,0,3115,46049972,4.6004e+07,1.08415e+06,0,4.60694e+07,4.61081e+07,4.61847e+07,46392556,-19.306,471.524
is_anonymous,bool,35001,0,2,True,0.728008,0.444992,False,0,1,1,True,-1.02483,-0.949777


In [189]:
# Outliers for amount column

# I chose to remove outliers first because of how high the max values were. This would put our theoretical min at a negative, and I didn't want the skewness to move us into negative values for later data analysis

# Tukey Box Plot

import pandas as pd
from matplotlib import pyplot as plt

# Create a DataFrame to store the output
outliers = pd.DataFrame(columns=['min', 'count below', 'max', 'count above'])

# Calculate the theoretical min/max based on 1.5
q1 = play_data['amount'].quantile(.25)
q3 = play_data['amount'].quantile(.75)
min = q1 - (1.5 * (q3 - q1))
max = q3 + (1.5 * (q3 - q1))

outliers.loc['amount'] = (min, play_data['amount'][play_data['amount'] < min].count(), max, play_data['amount'][play_data['amount'] > max].count())

# Let's take a look
outliers

,min,count below,max,count above
amount,-37.5,0.0,102.5,2506.0


In [195]:
# Then, let's throw this into a function to help with outliers
remove_outliers(play_data, max, min)

In [196]:
# I'll run the unistats again to see what we are looking at for skewness after this adjustment
uniStats(play_data)

# The skewness went down significantly from what it was. This is great news!! After this, we'll export

,DataType,Count,Null Count,Unique Values,Mode,Mean,STD,Min,25%,Median,75%,Max,Skew,Kurt
created_at,object,35001,0,33738,2020-03-12T17:21:45-05:00,-,-,-,-,-,-,-,-,-
amount,float64,35001,0,91,50,40.7201,33.3241,5,15,25,50,102.5,0.877035,-0.649268
donation_id,int32,35001,0,35001,31916,4.55054e+08,1.22785e+08,31916,4.87295e+08,4.8892e+08,4.89799e+08,496171348,-3.43001,9.78924
campaign_id,int32,35001,0,3115,46049972,4.6004e+07,1.08415e+06,0,4.60694e+07,4.61081e+07,4.61847e+07,46392556,-19.306,471.524
is_anonymous,bool,35001,0,2,True,0.728008,0.444992,False,0,1,1,True,-1.02483,-0.949777


In [197]:
# Let's export this to a CSV
play_data.to_csv(r'C:\Analytics\data\donations_clean.csv', index = False)

In [192]:
# Functions:

def remove_outliers(df, max, min):
    df.loc[df['amount'] > max, 'amount' ] = max

In [17]:
def uniStats(df):
    # Import packages
    import pandas as pd
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.max_columns', 100)

    # Build the data frame
    new_df = pd.DataFrame(columns = ['DataType', 'Count', 'Null Count', 'Unique Values', 'Mode', 
                                     'Mean', 'STD', 'Min', '25%', 'Median', '75%', 'Max', 'Skew', 'Kurt'])

    # Build the data frame:
    for col in df:
        # Build data frame for numeric stats:
        if pd.api.types.is_numeric_dtype(df[col]):
            new_df.loc[col] = [df[col].dtype, df[col].count(), df[col].isnull().sum(), df[col].nunique(), 
                               df[col].mode().values[0], df[col].mean(), df[col].std(), df[col].min(), 
                               df[col].quantile(.25), df[col].median(), df[col].quantile(.75), df[col].max(), 
                               df[col].skew(), df[col].kurt()]
        # Build data frame for categorical stats:
        else:
            new_df.loc[col] = [df[col].dtype, df[col].count(), df[col].isnull().sum(), df[col].nunique(), 
                               df[col].mode().values[0], '-', '-', '-', '-', '-', '-', '-', '-', '-']

    return new_df.sort_values(by=['DataType', 'Skew', 'Unique Values'], ascending=False)